In [1]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [2]:
from comm_agents.data.reference_experiments_with_qa import RefExperimentMass, RefExperimentCharge
from ipywidgets import interact
import numpy as  np

In [81]:
# experiment parameters
GOLF_HOLE_LOC_M = 0.1
GOLF_HOLE_LOC_C = 1
PARAM_DICT = dict(
    m=[2e-12, 1e-12],
    q=[3e-12, -4e-12],
    m_ref=2e-12,
    v_ref=1,
    N=100,
    alpha=[0, 0],
    dt=.01,
    d=.1,
    gravity=True)

In [82]:
alpha_range_re1 = np.linspace(0, 0.6, 20)
m_range_re1 = np.linspace(1e-12, 3e-12, 20)
alpha_range_re2 = np.linspace(0.5, 1, 20)
q_range_re2_1 = np.linspace(1e-12, 4e-12, 20)
q_range_re2_2 = np.linspace(-3e-12, -5e-12, 20)
N_reange = [10, 100, 1000, 10000, 100000]

In [83]:
def get_re1_plot(alpha1, alpha2, m1, m2, N):
    rem = RefExperimentMass(**PARAM_DICT)
    rem.N = N
    rem.set_initial_state()
    rem.alpha = np.array([alpha1*np.pi, alpha2*np.pi])
    rem.m = np.array([m1, m2])
    rem.run()
    rem.visualize(golf_hole_loc=GOLF_HOLE_LOC_M)
    
interact(get_re1_plot, alpha1=alpha_range_re1, alpha2=alpha_range_re1,
         m1=m_range_re1, m2=m_range_re1, N=N_reange)

interactive(children=(Dropdown(description='alpha1', options=(0.0, 0.031578947368421054, 0.06315789473684211, …

<function __main__.get_re1_plot(alpha1, alpha2, m1, m2, N)>

In [84]:
def get_re2_plot(alpha1, alpha2, q1, q2, N):
    req = RefExperimentCharge(**PARAM_DICT)
    req.N = N
    req.set_initial_state()
    req.alpha = np.array([alpha1*np.pi, alpha2*np.pi])
    req.q = np.array([q1, q2])
    req.run()
    req.visualize(golf_hole_loc=.1)
interact(get_re2_plot, alpha1=alpha_range_re2, alpha2=alpha_range_re2,
        q1=q_range_re2_1, q2=q_range_re2_2, N=N_reange)

interactive(children=(Dropdown(description='alpha1', options=(0.5, 0.5263157894736842, 0.5526315789473684, 0.5…

<function __main__.get_re2_plot(alpha1, alpha2, q1, q2, N)>

In [108]:
from comm_agents.data.optimal_answers import get_alpha_star

In [110]:
get_alpha_star(req)

2020-10-12 19:07:42.741 | WARNING  | comm_agents.data.optimal_answers:get_single_charge_loss_value:32 - Particle 0 did not cross line with x=0
2020-10-12 19:07:42.741 | WARNING  | comm_agents.data.optimal_answers:get_single_charge_loss_value:32 - Particle 1 did not cross line with x=0
2020-10-12 19:07:42.750 | WARNING  | comm_agents.data.optimal_answers:get_single_charge_loss_value:32 - Particle 1 did not cross line with x=0
2020-10-12 19:07:42.758 | WARNING  | comm_agents.data.optimal_answers:get_single_charge_loss_value:32 - Particle 0 did not cross line with x=0
2020-10-12 19:07:42.779 | WARNING  | comm_agents.data.optimal_answers:get_single_charge_loss_value:32 - Particle 1 did not cross line with x=0


0.004443657710424806


array([1.81604019, 1.84688712])

In [ ]:
get_alpha_star(req, .1, )

In [86]:
objective_golf([0.9*np.pi, 0.9*np.pi], rem, get_single_mass_loss_value, .1)

0.07030868261376619

In [87]:
objective_golf([0.55*np.pi, 0.55*np.pi], req, get_single_charge_loss_value, .1)

0.0034666944064437365

In [88]:
from scipy import optimize
initial_guess = [0.4999 * np.pi, 0.4999 * np.pi]
alpha_star = optimize.minimize(
    objective_golf,
    initial_guess,
    method='Nelder-Mead',
    args=(rem, get_single_mass_loss_value, .1),
).x

In [89]:
alpha_star / np.pi

array([0.28900965, 0.40777474])

In [90]:
rem = RefExperimentMass(**PARAM_DICT)

In [91]:
req = RefExperimentCharge(**PARAM_DICT)

In [92]:
initial_guess = [0.51 * np.pi, 0.51 * np.pi]
alpha_star = optimize.minimize(
    objective_golf,
    initial_guess,
    method='Nelder-Mead',
    args=(req, get_single_charge_loss_value, .1),
)

2020-10-12 18:43:23.313 | WARNING  | comm_agents.data.optimal_answers:get_single_charge_loss_value:24 - Particle 1 did not cross line with x=0
2020-10-12 18:43:23.321 | WARNING  | comm_agents.data.optimal_answers:get_single_charge_loss_value:24 - Particle 1 did not cross line with x=0


In [103]:
alpha_star

 final_simplex: (array([[1.81608535, 1.84685172],
       [1.81609962, 1.84676524],
       [1.81608248, 1.8467619 ]]), array([1.43585213e-05, 1.43591558e-05, 1.43615971e-05]))
           fun: 1.435852127548144e-05
       message: 'Optimization terminated successfully.'
          nfev: 89
           nit: 45
        status: 0
       success: True
             x: array([1.81608535, 1.84685172])

In [94]:
req.set_initial_state()
req.alpha = alpha_star.x
req.run()
req.visualize(.1)